To begin to develop the `kinisi` code, you should clone the git repository and install `kinisi` with the `dev` option in editable mode. 

```console
$ git clone https://github.com/bjmorgan/kinisi.git
$ cd kinisi
$ pip install -e '.[dev]'
```

# DiffusionAnalyzer and JumpDiffusionAnalyzer

Within `kinisi` the main classes the end user will interact with are `diffusion_analyzer` and `jump_diffusion_analyzer`. 

These are typically called with 3 main methods depending on the type of trajectory these are:

* `from_universe` - For MDAnalysis Universe
* `from_xdatcar` - For Pymatgen Structures
* `from_ase` - For ASE objects



For instance, `DiffusionAnalyzer.from_xdatcar` can be called as in the Diffusion Coeffcienct from a VASP file workbook `vasp_d.ipynb` via

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipp as sc
from pymatgen.io.vasp import Xdatcar
from kinisi.analyze import DiffusionAnalyzer
rng = np.random.RandomState(42)

params = {'specie': 'Li',
          'time_step': 2.0 * sc.Unit('fs'),
          'step_skip': 50 * sc.Unit('dimensionless'),
          }


params['dimension']= 'xy'

xd = Xdatcar('./example_XDATCAR.gz')
diff = DiffusionAnalyzer.from_xdatcar(xd, **params)

`DiffusionAnalyzer` is a subclass of `Analyzer` and so upon its initialization `Analyzer` is initialized first, `DiffusionAnalyzer` inherits from the `Analyzer`.

```python

class DiffusionAnalyzer(Analyzer):

```


The `from_` class method of the `DiffusionAnalyzer` calls the Analyzer class `_from_` method via a super(), i.e. within `from_universe`

```python

p = super()._from_universe(trajectory, specie, time_step, step_skip, dtype, dt, dimension, distance_unit,
                            specie_indices, masses, progress)

```

Within the `_from_` methods the respective parser is called for each frame in the trajectory, similarly to above there are 3 parser classes : 


* `MDAnalysis_parser` - For MDAnalysis Universe
* `PymatgenParser` - For Pymatgen Structures
* `ASE_parser` - For ASE objects


Each respective parser class is sub class of the full `Parser` class. 

```python 

class PymatgenParser(Parser):

```

**The parser indexes the correct molecules and particle coordinates for a timestep.**

**The Analyzer calls the parser for all the chosen time steps and calculates displacements.**

**The DiffusionAnalyzer then calculates the mean squared displacements (MSD) and variances for each time interval.**

A flowchart of the classes and inheritance can be seen below



![My Diagram](./diffusion_analyzer_flowchart.png)

# DiffusionAnalyzer.diffusion()

In order to calculate the diffusion coefficient 